In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
#Todos os beams terão a_0 = 1.
#Os restantes parâmetros estarão de acordo com os do artigo em relação ao a_0

#parametros do short beam:

W0_s = 0.04       #spot size
R_s = 0.12        #radius
lambda_s = 0.0165 #wavelength
z_r_s = np.pi * W0_s * W0_s / lambda_s

#parametros do thin beam

W0_t = 0.062      #spot size
lambda_t = 0.0165 #wavelength
L_t = 4.13        #length
z_r_t = np.pi * W0_t * W0_t / lambda_t
a_z_t = 1 / np.sqrt(1 + (L_t/(4 * z_r_t)) * (L_t/(4 * z_r_t)))

#parametros do wide beam

W0_w = 0.41     #spot size
R_w = 1.64      #radius
lambda_w = 0.11 #wavelength
L_w = 6.97      #length
z_r_w = np.pi * W0_w * W0_w / lambda_w
a_z_w = 1 / np.sqrt(1 + (L_w/(4 * z_r_w)) * (L_t/(4 * z_r_w)))

num_pon = 10000

In [ ]:
#gerar os eletroes
rng = np.random.default_rng()

l = 0

coord_t = L_t / 2 * rng.random(num_pon) - L_t / 4  # coordenada z do thin beam

coord_s = np.array([np.random.normal(0, 1 / np.sqrt(2) * R_s,
                       num_pon), np.random.normal(0, 1 / np.sqrt(2) * R_s,
                       num_pon)])

for i in tqdm(range(num_pon)):
    z_w = L_w * rng.random() - L_w/2               #coordenada z do wide beam
    x_w = 2 * R_w * rng.random() - R_w             #coordenada x do wide beam
    y_w = 2 * R_w * rng.random() - R_w             #coordenada y do wide beam

    crc = x_w * x_w + y_w * y_w

    if l == 0 and crc <= R_w * R_w:
        #eletroes_w
        coord_w_x = np.array([x_w])
        coord_w_y = np.array([y_w])
        coord_w_z = np.array([z_w])
        l = 1

    if crc <= R_w * R_w:  # para ser circular de raio 2
        coord_w_x_s = np.array([x_w])
        coord_w_y_s = np.array([y_w])
        coord_w_z_s = np.array([z_w])
        coord_w_x = np.concatenate((coord_w_x, coord_w_x_s), axis=0)
        coord_w_y = np.concatenate((coord_w_y, coord_w_y_s), axis=0)
        coord_w_z = np.concatenate((coord_w_z, coord_w_z_s), axis=0)

    coord_w = np.array([coord_w_x, coord_w_y, coord_w_z])

In [ ]:
#gerar a amostragem
n_ele_s = len(coord_s)

n_ele_t = len(coord_t)

n_ele_w = len(coord_w)

delta_a = np.array([i/num_pon + 0.01 for i in range(num_pon)])

a_eff_s = np.exp(-(coord_s[0]**2 + coord_s[1]**2)/(W0_s * W0_s))  #vetor potencial do short beam

a_eff_t = 1 / np.sqrt(1 + (coord_t * coord_t)/(z_r_t*z_r_t))  #vetor potencial do thin beam


a_eff_w = np.exp((-(coord_w[0]*coord_w[0] + coord_w[1] * coord_w[1]))/(1 + (coord_w[2] * coord_w[2])/(z_r_w*z_r_w))/(W0_w * W0_w))/np.sqrt(1 + (coord_w[2] * coord_w[2])/(z_r_w*z_r_w)) #vetor potencial do wide beam

In [ ]:
#gerar curvas teoricas

func_t = np.zeros(num_pon)

func_w = np.zeros(num_pon)

func_s = (((n_ele_s * W0_s * W0_s)/(delta_a * R_s * R_s))*((delta_a)**((W0_s * W0_s)/(R_s * R_s))))  #função analítica do short beam

for i in tqdm(range(num_pon)):
    if delta_a[i] > a_z_t:
        func_t[i] = 10**-4 * (4 * n_ele_t * z_r_t * (1/L_t) * (1 / ((delta_a[i]) * (delta_a[i]))) * (1 / np.sqrt(1 - (delta_a[i]) * (delta_a[i]))))   #função analítica do short beam
    else:
        func_t[i] = 0


for i in tqdm(range(num_pon)):
    if delta_a[i] > a_z_w:
        func_w[i] = 0.035 * (4 * np.pi * W0_w * W0_w * z_r_w) / (3 * delta_a[i] * delta_a[i]) * np.sqrt(1 - delta_a[i] * delta_a[i]) * (2 + (1 / (delta_a[i] * delta_a[i])))  #função analítica do wide beam
    else:
        func_w[i] = 0.035 * (4 * np.pi * W0_w * W0_w * z_r_w * L_w) / (4 * delta_a[i] * z_r_w) * (1 + (L_w * L_w)/(16 * z_r_w))

In [ ]:
figure, axis = plt.subplots(3, 1)

figure.suptitle('Particle Distribution for Different Beams')

figure.tight_layout(pad=1.5)

axis[0].set_xlim([0, 1])
axis[1].set_xlim([0, 1])
axis[2].set_xlim([0, 1])

axis[0].set_xticks([0, 0.25, 0.5, 0.75, 1])

axis[1].set_xticks([0, 0.25, 0.5, 0.75, 1])

axis[2].set_xticks([0, 0.25, 0.5, 0.75, 1])

axis[0].hist(a_eff_s, 100, density=True, facecolor='b', alpha=0.75) #histograma do short beam

axis[1].hist(a_eff_t, 100, density=True, facecolor='r', alpha=0.75) #histograma do thin beam

axis[2].hist(a_eff_w, 100, density=True, facecolor='g', alpha=0.75) #histograma do wide beam

axis[0].set_title('Short beam')
axis[1].set_title('Thin beam')
axis[2].set_title('Wide beam')
axis[0].plot(delta_a, func_s)
axis[1].plot(delta_a, func_t)
axis[2].plot(delta_a, func_w)
axis[0].set_yscale('log')
axis[1].set_yscale('log')
axis[2].set_yscale('log')

plt.show()